In [38]:
import numpy as np
import pandas as pd
import plotly.express as px
import statsmodels.formula.api as smf


In [39]:
path = "~/Desktop/ISLR_Labs/data/Boston.csv"
data = pd.read_csv(path, index_col=0)
data.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000,50.000000


In [40]:
result_zn = smf.ols("crim ~ zn", data=data).fit()
result_indus = smf.ols("crim ~ indus", data=data).fit()
result_chas = smf.ols("crim ~ chas", data=data).fit()
result_nox = smf.ols("crim ~ nox", data=data).fit()
result_rm = smf.ols("crim ~ rm", data=data).fit()
result_age = smf.ols("crim ~ age", data=data).fit()
result_dis = smf.ols("crim ~ dis", data=data).fit()
result_rad = smf.ols("crim ~ rad", data=data).fit()
result_tax = smf.ols("crim ~ tax", data=data).fit()
result_ptratio = smf.ols("crim ~ ptratio", data=data).fit()
result_lstat = smf.ols("crim ~ lstat", data=data).fit()
result_medv = smf.ols("crim ~ medv", data=data).fit()

In [41]:
columns_name = data.columns[1::]

values = np.array([None] * 2 * len(columns_name))
values = values.reshape(len(columns_name), 2)
for i, name in enumerate(columns_name):
    result_name = "result_" + name
    value = np.array([eval(result_name).pvalues[1], eval(result_name).params[1]])
    values[i] = value

df_p_values = pd.DataFrame(values, index=columns_name, dtype=np.float64,
                           columns=["p_value", "coefficient"]).sort_index()
df_p_values

,p_value,coefficient
age,2.854869e-16,0.107786
chas,2.094345e-01,-1.892777
dis,8.519949e-19,-1.550902
indus,1.450349e-21,0.509776
lstat,2.654277e-27,0.548805
medv,1.173987e-19,-0.363160
nox,3.751739e-23,31.248531
ptratio,2.942922e-11,1.151983
rad,2.693844e-56,0.617911
rm,6.346703e-07,-2.684051


- For all predictors except for **chas** the coefficient are statistically significant
- The relationship is positive for **indus**, **nox**, **age**, **rad**, **tax**, **ptratio** and **lstat**

In [42]:
fig = px.scatter(data, x="ptratio", y="crim")
x = data.ptratio
y = result_ptratio.params[1] * x + result_ptratio.params[0]
fig.add_scatter(x=x, y=y, name="trendline")

In [43]:
fig = px.scatter(data, x="medv", y="crim")
x = data.medv
y = result_medv.params[1] * x + result_medv.params[0]
fig.add_scatter(x=x, y=y, name="trendline")

b)

In [44]:
all_columns = " + ".join(data.columns.difference(["crim"]))
formula = 'crim ~ ' + all_columns
results = smf.ols(formula, data=data).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   crim   R-squared:                       0.449
Model:                            OLS   Adj. R-squared:                  0.436
Method:                 Least Squares   F-statistic:                     33.52
Date:                Fri, 05 May 2023   Prob (F-statistic):           2.03e-56
Time:                        21:22:48   Log-Likelihood:                -1655.4
No. Observations:                 506   AIC:                             3337.
Df Residuals:                     493   BIC:                             3392.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.7784      7.082      1.946      0.0

- The coefficients and their statistical significancy have changed
- The null hypothesis cannot be rejected by predictors **age**, **chas**, **indus**, **ptratio**, **rm**, **tax**

c)

In [45]:
px.scatter(x=df_p_values.coefficient, y=results.params[1:],
           labels={"x": "Univariate regression coefficients",
                   "y": "Multiple regression coefficients"},
           color=df_p_values.index)

d)

In [46]:
result_poly_zn = smf.ols("crim ~ zn + I(zn ** 2) + I(zn ** 3)", data=data).fit()
result_poly_indus = smf.ols("crim ~ indus + I(indus ** 2) + I(indus ** 3)", data=data).fit()
result_poly_chas = smf.ols("crim ~ chas + I(chas ** 2) + I(chas ** 3)", data=data).fit()
result_poly_nox = smf.ols("crim ~ nox + I(nox ** 2) + I(nox ** 3)", data=data).fit()
result_poly_rm = smf.ols("crim ~ rm + I(rm ** 2) + I(rm ** 3)", data=data).fit()
result_poly_age = smf.ols("crim ~ age + I(age ** 2) + I(age ** 3)", data=data).fit()
result_poly_dis = smf.ols("crim ~ dis + I(dis ** 2) + I(dis ** 3)", data=data).fit()
result_poly_rad = smf.ols("crim ~ rad + I(rad ** 2) + I(rad ** 3)", data=data).fit()
result_poly_tax = smf.ols("crim ~ tax + I(tax ** 2) + I(tax ** 3)", data=data).fit()
result_poly_ptratio = smf.ols("crim ~ ptratio + I(ptratio ** 2) + I(ptratio ** 3)", data=data).fit()
result_poly_lstat = smf.ols("crim ~ lstat + I(lstat ** 2) + I(lstat ** 3)", data=data).fit()
result_poly_medv = smf.ols("crim ~ medv + I(medv ** 2) + I(medv ** 3)", data=data).fit()

In [47]:
values = np.array([None] * 3 * len(columns_name))
values = values.reshape(len(columns_name), 3)
for i, name in enumerate(columns_name):
    result_name = "result_poly_" + name
    value = np.array([eval(result_name).pvalues[1], eval(result_name).pvalues[2], eval(result_name).pvalues[3]])
    values[i] = value

df_poly_p_values = pd.DataFrame(values, index=columns_name, dtype=np.float64,
                           columns=["degree 1", "degree 2", "degree 3"])
df_poly_p_values

,degree 1,degree 2,degree 3
zn,2.612296e-03,9.375050e-02,2.295386e-01
indus,5.297064e-05,3.420187e-10,1.196405e-12
chas,2.094345e-01,2.094345e-01,2.094345e-01
nox,2.758372e-13,6.811300e-15,6.961110e-16
rm,2.117564e-01,3.641094e-01,5.085751e-01
age,1.426608e-01,4.737733e-02,6.679915e-03
dis,6.374792e-18,4.941214e-12,1.088832e-08
rad,6.234175e-01,6.130099e-01,4.823138e-01
tax,1.097075e-01,1.374682e-01,2.438507e-01
ptratio,3.028663e-03,4.119552e-03,6.300514e-03


- For degree one **zn**, **indus**, **nox**, **dis**, **ptratio**, **medv** are statistically significant
- For degree two **zn**, **indus**, **nox**, **age**, **dis**, **ptratio**, **lstat**, **medv** are statistically significant
- For degree three **indus**, **nox**, **age**, **dis**, **ptratio**, **medv** are statistically significant
- This show some evidence that there is some non-linearity in the data